In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import nbimporter
from functions import *
from numpy import *
import matplotlib.pyplot as plt
%matplotlib inline

import os
import subprocess

# 1- make structure for files

In [ ]:
if not os.path.exists('/home/mahdi/Desktop/valid'):
     os.makedirs('/home/mahdi/Desktop/valid')

for i in range(1,82):
    os.makedirs('/home/mahdi/Desktop/valid/'+str(i))
    
    os.makedirs('/home/mahdi/Desktop/valid/'+str(i)+'/main_seg')
    os.makedirs('/home/mahdi/Desktop/valid/'+str(i)+'/sct_seg')
    os.makedirs('/home/mahdi/Desktop/valid/'+str(i)+'/my_seg/first_ref')
    os.makedirs('/home/mahdi/Desktop/valid/'+str(i)+'/my_seg/zero_ref')
    os.makedirs('/home/mahdi/Desktop/valid/'+str(i)+'/my_seg/mean_ref')



# 2- resize main data to valid folder

In [ ]:
def resize (file_path: str,output_direction):
    if not (file_path.endswith(".nii") or file_path.endswith(".nii.gz")):
        raise ValueError(
              f"Nifti file path must end with .nii or .nii.gz, got {file_path}."
                        )
    img = nib.load(file_path)
    img_data = img.get_fdata()
    img_data = img_data[23:87,23:87,:,:]
    
    header=img.header  
    ## edit the header for shape
    header['dim'][1:5]=img_data.shape
    img_mask_affine = img.affine
    
    img_reshape = nib.Nifti1Image(img_data, affine=img_mask_affine, header=header)
    return nib.save(img_reshape,output_direction) 

In [2]:
D7_dir='/home/mahdi/Desktop/data_selection_D7'
n,name=count(D7_dir)

In [2]:
out_dir='/home/mahdi/Desktop/valid'
n1,name1=count(out_dir)

In [3]:
name1

array([['76'],
       ['70'],
       ['13'],
       ['52'],
       ['32'],
       ['16'],
       ['30'],
       ['39'],
       ['49'],
       ['73'],
       ['21'],
       ['55'],
       ['35'],
       ['27'],
       ['66'],
       ['51'],
       ['14'],
       ['5'],
       ['81'],
       ['75'],
       ['61'],
       ['60'],
       ['63'],
       ['59'],
       ['77'],
       ['7'],
       ['9'],
       ['48'],
       ['56'],
       ['22'],
       ['33'],
       ['44'],
       ['41'],
       ['26'],
       ['36'],
       ['24'],
       ['23'],
       ['46'],
       ['45'],
       ['74'],
       ['57'],
       ['72'],
       ['2'],
       ['62'],
       ['15'],
       ['47'],
       ['43'],
       ['18'],
       ['17'],
       ['42'],
       ['19'],
       ['69'],
       ['64'],
       ['10'],
       ['65'],
       ['71'],
       ['78'],
       ['54'],
       ['3'],
       ['38'],
       ['58'],
       ['20'],
       ['79'],
       ['11'],
       ['80'],
       ['6'],
       ['12'],
 

In [4]:
name[10]

array(['S24_RND.nii.gz'], dtype='<U17')

In [5]:
name1[10]

array(['21'], dtype='<U2')

In [ ]:
for i in range(n):
    resize(D7_dir+'/'+name[i][0],out_dir+'/'+name1[i][0]+'/'+'main_seg'+'/'+name[i][0])

# 3- make mean data

In [ ]:
for j in range(20,81):
    data_dir=out_dir+'/'+name1[j][0]+'/'+'main_seg'+'/'+name[j][0]
    out='/home/mahdi/Desktop/valid/'+name1[j][0]+'/main_seg/mean.nii'
    
    
    subprocess.Popen(['sct_maths','-i',data_dir,'-mean','t' ,'-o',out ]) 


# 4- get result from my algo to related folder

### mean ref

In [ ]:
maximum_intensity=1705
model='200epoch_with_val.h5'

for i in range(40,n):
    
    x=(len(name[i][0][:])-7)
    output_direction_mean='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/mean_ref/'+name[i][0][:x]+'_plus.'+name[i][0][-6:]
    input_direction=out_dir+'/'+name1[i][0]+'/'+'main_seg'+'/'+name[i][0]
    reference='/home/mahdi/Desktop/valid/'+name1[i][0]+'/main_seg/mean.nii'
    
    main(input_direction,reference,output_direction_mean,maximum_intensity,model)

   
    

### zero ref 

In [ ]:
for i in range(n):
    
    x=(len(name[i][0][:])-7)
    output_direction_mean='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/zero_ref/'+name[i][0][:x]+'_plus.'+name[i][0][-6:]
    input_direction=out_dir+'/'+name1[i][0]+'/'+'main_seg'+'/'+name[i][0]
    reference='0'
    
    main(input_direction,reference,output_direction_mean,maximum_intensity,model)

In [ ]:
i=8
maximum_intensity=1705
model='200epoch_with_val.h5'

x=(len(name[i][0][:])-7)
output_direction_mean='/home/mahdi/Desktop/'+name[i][0][:x]+'_plus_mid.'+name[i][0][-6:]
input_direction=out_dir+'/'+name1[i][0]+'/'+'main_seg'+'/'+name[i][0]
reference='101'

main(input_direction,reference,output_direction_mean,maximum_intensity,model)

In [ ]:
img = nib.load(input_direction)
img.shape

In [ ]:
i=8
maximum_intensity=1705
model='200epoch_with_val.h5'

x=(len(name[i][0][:])-7)
output_direction_mean='/home/mahdi/Desktop/'+name[i][0][:x]+'_plus_zero_mean.'+name[i][0][-6:]
input_direction=out_dir+'/'+name1[i][0]+'/'+'main_seg'+'/'+name[i][0]
reference='/home/mahdi/Desktop/valid/'+name1[i][0]+'/main_seg/mean.nii'

main(input_direction,reference,output_direction_mean,maximum_intensity,model)

In [ ]:
name[8][0]

### mid ref 

In [ ]:
for i in range(n):
    
    x=(len(name[i][0][:])-7)
    output_direction_mean='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/first_ref/'+name[i][0][:x]+'_plus.'+name[i][0][-6:]
    input_direction=out_dir+'/'+name1[i][0]+'/'+'main_seg'+'/'+name[i][0]
    y=int(nib.load(input_direction).shape[3]/2)
    reference=str(y)
    
    main(input_direction,reference,output_direction_mean,maximum_intensity,model)

In [ ]:
i=8
maximum_intensity=1705
model='200epoch_with_val_landa_0.01.h5'
x=(len(name[i][0][:])-7)
output_direction_mean='/home/mahdi/Desktop/'+name[i][0][:x]+'_plus.'+name[i][0][-6:]
input_direction=out_dir+'/'+name1[i][0]+'/'+'main_seg'+'/'+name[i][0]
y=int(nib.load(input_direction).shape[3]/2)
reference=str(y)

main(input_direction,reference,output_direction_mean,maximum_intensity,model)

# 5- resize sct data

In [ ]:
for j in range(n):
    input_dir='/home/mahdi/Desktop/valid/'+name1[j][0]+'/sct_seg/fmri_rmvol_moco.nii.gz'
    out_dir='/home/mahdi/Desktop/valid/'+name1[j][0]+'/sct_seg/fmri_rmvol_moco.nii.gz'
    resize(input_dir,out_dir)

# 6- make TSNR data

### For main

In [ ]:
for j in range(20,n):
    data_dir=out_dir+'/'+name1[j][0]+'/'+'main_seg'+'/'+name[j][0]
    out='/home/mahdi/Desktop/valid/'+name1[j][0]+'/main_seg/tsnr.nii'
    
    
    subprocess.Popen(['sct_fmri_compute_tsnr','-i',data_dir,'-o',out ]) 

### For mine 

In [ ]:
for j in range(n):
    
    x=(len(name[j][0][:])-7)
    data_dir='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/mean_ref/'+name[j][0][:x]+'_plus.'+name[j][0][-6:]
    
    out='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/mean_ref/tsnr.nii'
    
    
    subprocess.Popen(['sct_fmri_compute_tsnr','-i',data_dir,'-o',out ]) 

In [ ]:
for j in range(n):
    
    x=(len(name[j][0][:])-7)
    data_dir='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/zero_ref/'+name[j][0][:x]+'_plus.'+name[j][0][-6:]
    
    out='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/zero_ref/tsnr.nii'
    
    
    subprocess.Popen(['sct_fmri_compute_tsnr','-i',data_dir,'-o',out ]) 

In [ ]:
for j in range(n):
    
    x=(len(name[j][0][:])-7)
    data_dir='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/first_ref/'+name[j][0][:x]+'_plus.'+name[j][0][-6:]
    
    out='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/first_ref/tsnr.nii'
    
    
    subprocess.Popen(['sct_fmri_compute_tsnr','-i',data_dir,'-o',out ]) 

### For sct

In [ ]:
for j in range(n):
    data_dir=out_dir+'/'+name1[j][0]+'/'+'sct_seg'+'/'+'fmri_rmvol_moco.nii.gz'
    out='/home/mahdi/Desktop/valid/'+name1[j][0]+'/sct_seg/tsnr.nii'
    
    
    subprocess.Popen(['sct_fmri_compute_tsnr','-i',data_dir,'-o',out ]) 

# 7- resize and copy segmentation file

In [ ]:
def resize_seg (file_path: str,output_direction):
    if not (file_path.endswith(".nii") or file_path.endswith(".nii.gz")):
        raise ValueError(
              f"Nifti file path must end with .nii or .nii.gz, got {file_path}."
                        )
    img = nib.load(file_path)
    img_data = img.get_fdata()
    img_data = img_data[23:87,23:87,:]
    
    header=img.header  
    ## edit the header for shape
    header['dim'][1:4]=img_data.shape
    img_mask_affine = img.affine
    
    img_reshape = nib.Nifti1Image(img_data, affine=img_mask_affine, header=header)
    return nib.save(img_reshape,output_direction) 

### resize csf part 

In [ ]:
for i in range(n):
    
    data_dir1=out_dir+'/'+name1[i][0]+'/'+'/'+'mask_seg2_Dil_CSF.nii.gz'
    
    output1='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_part.nii'
    output2='/home/mahdi/Desktop/valid/'+name1[i][0]+'/main_seg/csf_part.nii'
    output3_1='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/zero_ref/csf_part.nii'
    output3_2='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/first_ref/csf_part.nii'
    output3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/mean_ref/csf_part.nii'

    resize_seg(data_dir1,output1)
    resize_seg(data_dir1,output2)
    resize_seg(data_dir1,output3_1)
    resize_seg(data_dir1,output3_2)
    resize_seg(data_dir1,output3_3)


### resize spine part

In [ ]:
for i in range(n):
    
    data_dir2=out_dir+'/'+name1[i][0]+'/'+'/'+'tmp_t2s_seg_reg.nii.gz'
    
    output4='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_part.nii'
    output5='/home/mahdi/Desktop/valid/'+name1[i][0]+'/main_seg/spine_part.nii'
    output6_1='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/zero_ref/spine_part.nii'
    output6_2='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/first_ref/spine_part.nii'
    output6_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/mean_ref/spine_part.nii'   
    
    resize_seg(data_dir2,output4)
    resize_seg(data_dir2,output5)
    resize_seg(data_dir2,output6_1)
    resize_seg(data_dir2,output6_2)
    resize_seg(data_dir2,output6_3)        

# 8- edit all segmentation 

# 9- create mask 

### sct mask

In [ ]:
for i in range(n):
    second_1='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/tsnr.nii'

    
    input1='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_part.nii'
    output1='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_mask.nii'
    subprocess.Popen(['sct_maths','-i',input1,'-mul',second_1 ,'-o',output1]) 
    
    input1='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_part.nii'
    output1='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_mask.nii'
    subprocess.Popen(['sct_maths','-i',input1,'-mul',second_1 ,'-o',output1]) 
    

### main mask

In [ ]:
for i in range(n):
    second_2='/home/mahdi/Desktop/valid/'+name1[i][0]+'/main_seg/tsnr.nii' 

    input2='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_part.nii'
    output2='/home/mahdi/Desktop/valid/'+name1[i][0]+'/main_seg/csf_mask.nii'
    subprocess.Popen(['sct_maths','-i',input2,'-mul',second_2 ,'-o', output2])
    
    input2='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_part.nii'
    output2='/home/mahdi/Desktop/valid/'+name1[i][0]+'/main_seg/spine_mask.nii'
    subprocess.Popen(['sct_maths','-i',input2,'-mul',second_2 ,'-o', output2])

## my mask 

### zero_ref

In [ ]:
for i in range(40,n):
    second3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/zero_ref/tsnr.nii'

    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_part.nii'
    output3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/zero_ref/csf_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 
    
    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_part.nii'
    output3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/zero_ref/spine_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 
    

### mid_ref 

In [ ]:
for i in range(40,n):
    second3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/first_ref/tsnr.nii'

    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_part.nii'
    output3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/first_ref/csf_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 
    
    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_part.nii'
    output3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/first_ref/spine_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 
    

### mean_ref 

In [ ]:
for i in range(40,n):
    second3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/mean_ref/tsnr.nii'

    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_part.nii'
    output3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/mean_ref/csf_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 
    
    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_part.nii'
    output3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/mean_ref/spine_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 
    

# 10-Tsnr mean and compare result

In [12]:
csf_main1=[]
csf_sct1 =[]
csf_mine1=[]
csf_mine2=[]
csf_mine3=[]
spine_main1=[]
spine_mine1=[]
spine_mine2=[]
spine_mine3=[]
spine_sct1 =[]
start_time=time.time()

for i in range(n):
    output_sct_csf='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_mask.nii'
    output_main_csf='/home/mahdi/Desktop/valid/'+name1[i][0]+'/main_seg/csf_mask.nii'
    output_mine_zeroref_csf='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/zero_ref/csf_mask.nii'
    
    output_mine_midref_csf='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/first_ref/csf_mask.nii'
    output_mine_meanref_csf='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/mean_ref/csf_mask.nii'

    
    
    
    output_sct_spine='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_mask.nii'
    output_main_spine='/home/mahdi/Desktop/valid/'+name1[i][0]+'/main_seg/spine_mask.nii'
    output_mine_zeroref_spine='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/zero_ref/spine_mask.nii' 
    
    output_mine_midref_spine='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/first_ref/spine_mask.nii' 
    output_mine_meanref_spine='/home/mahdi/Desktop/valid/'+name1[i][0]+'/my_seg/mean_ref/spine_mask.nii' 

    
    
    
    csf_main=mean_all(output_main_csf)
    csf_main1.append(csf_main)
    csf_sct=mean_all(output_sct_csf)
    csf_sct1.append(csf_sct)
    
    csf_mine=mean_all(output_mine_zeroref_csf)
    csf_mine1.append(csf_mine)
    csf_mine_mid=mean_all(output_mine_midref_csf)
    csf_mine2.append(csf_mine_mid)
    csf_mine_mean=mean_all(output_mine_meanref_csf)
    csf_mine3.append(csf_mine_mean)
    
    
    spine_main=mean_all(output_main_spine)
    spine_main1.append(spine_main)
    spine_sct=mean_all(output_sct_spine)
    spine_sct1.append(spine_sct)
    
    
    spine_mine=mean_all(output_mine_zeroref_spine)
    spine_mine1.append(spine_mine)
    spine_mine_mid=mean_all(output_mine_midref_spine)
    spine_mine2.append(spine_mine_mid)
    spine_mine_mean=mean_all(output_mine_meanref_spine)
    spine_mine3.append(spine_mine_mean)

    
    

csf_main=np.mean(csf_main1)
csf_sct=np.mean(csf_sct1)
csf_mine_zero=np.mean(csf_mine1)
csf_mine_mid=np.mean(csf_mine2)
csf_mine_mean=np.mean(csf_mine3)




spine_main=np.mean(spine_main1)
spine_sct=np.mean(spine_sct1)
spine_mine_zero=np.mean(spine_mine1)
spine_mine_mid=np.mean(spine_mine2)
spine_mine_mean=np.mean(spine_mine3)

print("--- %s second ---" % (time.time() - start_time))

    
pd.DataFrame([csf_main,csf_sct,csf_mine_zero,csf_mine_mid,csf_mine_mean,spine_main,spine_sct,spine_mine_zero,spine_mine_mid,spine_mine_mean],
         index=['main_csf_tsnr', 'sct_csf_tsnr', 'my_csf_tsnr_first','my_csf_tsnr_mid','my_csf_tsnr_mean',
                'main_spine_tsnr','sct_spine_tsnr','my_spine_tsnr_first','my_spine_tsnr_mid','my_spine_tsnr_mean']
        )

--- 369.800829410553 second ---


,0
main_csf_tsnr,4.038673
sct_csf_tsnr,7.146877
my_csf_tsnr_first,7.989082
my_csf_tsnr_mid,8.317330
my_csf_tsnr_mean,8.828488
main_spine_tsnr,7.104279
sct_spine_tsnr,12.899953
my_spine_tsnr_first,12.603740
my_spine_tsnr_mid,12.411145
my_spine_tsnr_mean,13.092095


In [ ]:
(4.038673+7.104279)/2

In [ ]:
print(p.shape)
64*64

In [ ]:
final=np.array([csf_main1 , csf_sct1,csf_mine1,csf_mine2,csf_mine3,
                spine_main1,spine_sct1,spine_mine1,spine_mine2,spine_mine3])
pd.DataFrame(final,index=['csf_nomoco','csf_sct','csf_myalgo_zero_ref','csf_myalgo_mid_ref','csf_myalgo_mean_ref','spine_nomoco','spine_sct','spine_myalgo_zero_ref','spine_myalgo_mid_ref','spine_myalgo_mean_ref']).to_csv('/home/mahdi/Desktop/result/tsnr_result.csv')

# 11- calculate Dvars

### for main 

In [ ]:
for j in range(n):
    data_dir=out_dir+'/'+name1[j][0]+'/'+'main_seg'+'/'+name[j][0]
    out='/home/mahdi/Desktop/valid/'+name1[j][0]+'/main_seg'
    subprocess.Popen(['fsl_motion_outliers','-i',data_dir,'-s',out+'/Dvars','-o',out+'/e' ]) 

### for sct

In [ ]:
for j in range(n):
    data_dir=out_dir+'/'+name1[j][0]+'/'+'sct_seg'+'/'+'fmri_rmvol_moco.nii.gz'
    out='/home/mahdi/Desktop/valid/'+name1[j][0]+'/sct_seg'
    subprocess.Popen(['fsl_motion_outliers','-i',data_dir,'-s',out+'/Dvars','-o',out+'/e' ]) 

### for mine 

In [ ]:
for j in range(n):
    
    x=(len(name[j][0][:])-7)
    data_dir='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/mean_ref/'+name[j][0][:x]+'_plus.'+name[j][0][-6:]
    out='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/mean_ref'
    subprocess.Popen(['fsl_motion_outliers','-i',data_dir,'-s',out+'/Dvars','-o',out+'/e' ]) 

In [ ]:
for j in range(n):
    
    x=(len(name[j][0][:])-7)
    data_dir='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/zero_ref/'+name[j][0][:x]+'_plus.'+name[j][0][-6:]
    out='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/zero_ref'
    subprocess.Popen(['fsl_motion_outliers','-i',data_dir,'-s',out+'/Dvars','-o',out+'/e' ]) 

In [ ]:
for j in range(n):
    
    x=(len(name[j][0][:])-7)
    data_dir='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/first_ref/'+name[j][0][:x]+'_plus.'+name[j][0][-6:]
    out='/home/mahdi/Desktop/valid/'+name1[j][0]+'/my_seg/first_ref'
    subprocess.Popen(['fsl_motion_outliers','-i',data_dir,'-s',out+'/Dvars','-o',out+'/e' ]) 

# 12- Dvars 

### main


In [5]:
main_Dvars=[]
for j in range(n):
    data_dir=out_dir+'/'+name1[j][0]+'/'+'main_seg'+'/Dvars'
    main_Dvars.append(np.mean(pd.read_csv(data_dir)))
mean_main_Dvars=np.mean(main_Dvars) 
pd.DataFrame(main_Dvars)  
#pd.DataFrame(main_Dvars).to_csv('/home/mahdi/Desktop/result/main_Dvars.csv')

,0
0,0.033274
1,0.027648
2,0.022165
3,0.044346
4,0.044644
...,...
76,0.052032
77,0.020753
78,0.028959
79,0.031971


### sct

In [6]:
sct_Dvars=[]
for j in range(n):
    data_dir=out_dir+'/'+name1[j][0]+'/'+'sct_seg'+'/Dvars'
    sct_Dvars.append(np.mean(pd.read_csv(data_dir)))
mean_sct_Dvars=np.mean(sct_Dvars)   
pd.DataFrame(sct_Dvars)  
#pd.DataFrame(sct_Dvars).to_csv('/home/mahdi/Desktop/result/sct_Dvars.csv')

,0
0,0.035907
1,0.025350
2,0.019106
3,0.042239
4,0.041873
...,...
76,0.049396
77,0.021202
78,0.028252
79,0.025799


## my algo

### zero ref

In [7]:
my_zero_Dvars=[]
for j in range(n):
    data_dir=out_dir+'/'+name1[j][0]+'/my_seg/zero_ref/Dvars'
    my_zero_Dvars.append(np.mean(pd.read_csv(data_dir)))
mean_my_zero_Dvars=np.mean(my_zero_Dvars)
pd.DataFrame(my_zero_Dvars)  
#pd.DataFrame(my_zero_Dvars).to_csv('/home/mahdi/Desktop/result/my_zero_Dvars.csv')

,0
0,0.023267
1,0.022712
2,0.019411
3,0.033088
4,0.035725
...,...
76,0.034692
77,0.017572
78,0.027254
79,0.028584


### mid_ref

In [8]:
my_mid_Dvars=[]
for j in range(n):
    data_dir=out_dir+'/'+name1[j][0]+'/my_seg/first_ref/Dvars'
    my_mid_Dvars.append(np.mean(pd.read_csv(data_dir)))
mean_my_mid_Dvars=np.mean(my_mid_Dvars)
pd.DataFrame(my_mid_Dvars)  
#pd.DataFrame(my_mid_Dvars).to_csv('/home/mahdi/Desktop/result/my_mid_Dvars.csv')

,0
0,0.023435
1,0.021614
2,0.020340
3,0.033807
4,0.038333
...,...
76,0.037197
77,0.015802
78,0.024575
79,0.027589


### mean ref 

In [9]:
my_mean_Dvars=[]
for j in range(n):
    data_dir=out_dir+'/'+name1[j][0]+'/my_seg/mean_ref/Dvars'
    my_mean_Dvars.append(np.mean(pd.read_csv(data_dir)))
mean_my_mean_Dvars=np.mean(my_mean_Dvars)
#pd.DataFrame(my_mean_Dvars)  
#pd.DataFrame(my_mean_Dvars).to_csv('/home/mahdi/Desktop/result/my_mean_Dvars.csv')

## Dvars result

In [10]:
pd.DataFrame([mean_main_Dvars,mean_sct_Dvars,mean_my_zero_Dvars,mean_my_mid_Dvars,mean_my_mean_Dvars],
            index=['mean_main_Dvars', 'mean_sct_Dvars', 'mean_my_zero_Dvars','mean_my_mid_Dvars','mean_my_mean_Dvars',
                ])

,0
mean_main_Dvars,0.034251
mean_sct_Dvars,0.031623
mean_my_zero_Dvars,0.026921
mean_my_mid_Dvars,0.027148
mean_my_mean_Dvars,0.024749


In [11]:
final_Dvars=np.array([main_Dvars ,sct_Dvars ,my_zero_Dvars ,my_mid_Dvars ,my_mean_Dvars])
final_Dvars=final_Dvars[:,:,0]
pd.DataFrame(final_Dvars,index=['main_Dvars','sct_Dvars','myalgo_zero_Dvars','myalgo_mid_Dvars','myalgo_mean_Dvars']).to_csv('/home/mahdi/Desktop/result/Dvars_result.csv')
